## Twin Analysis

The HCP databases uses a twin sample, allowing for the comparison of monozygotic (MZ) vs dizygotic (DZ) twins. 

This allows us to compare the heritability of oscillations.

In [1]:
# Import required libraries/functions
from __future__ import print_function, division
import sys
import random
import itertools
import numpy as  np

# Import custom code from module om, including general functions and OO code for handling data
sys.path.append('/Users/thomasdonoghue/Documents/GitCode/omegamappin/')

from om.core.db import OMDB
from om.core.osc import Osc
from om.core.utils import check_file_status
from om.meg.twin import *

In [2]:
# Pull out twin data from demographics file
mz_twins, dz_twins, all_twins, not_twins = get_twin_data()

# Match twin pairs
mz_id_pairs, mz_singles = match_twins(mz_twins)
dz_id_pairs, dz_singles = match_twins(dz_twins)

# Check how many pairs where extracted
print('There are', str(len(mz_id_pairs)), 'MZ twin pairs.')
print('There are', str(len(dz_id_pairs)), 'DZ twin pairs.')

There are 18 MZ twin pairs.
There are 19 DZ twin pairs.


In [3]:
# Get database object and set up database to use
db = OMDB()
dat_source = 'HCP'

# Check which twin subjects are available
av_dat, no_dat = check_file_status(all_twins, db, dat_source='HCP')

Of 93 subjects, 60 files are available and 33 files are unavailable.


In [4]:
# Given the missing data, check which twin pairs are complete
mz_complete_pairs = check_complete_pairs(mz_id_pairs, av_dat)
dz_complete_pairs = check_complete_pairs(dz_id_pairs, av_dat)

# Print out number of available pairs by twin type
print('There are', str(len(mz_complete_pairs)), 'complete MZ twin pairs.')
print('There are', str(len(dz_complete_pairs)), 'complete MZ twin pairs.')

There are 5 complete MZ twin pairs.
There are 11 complete MZ twin pairs.


In [5]:
# Get a list of all possible non-twin pairs, to compare to
all_pairs = list(itertools.combinations(av_dat, 2))
non_twin_pairs = rm_twin_pairs(all_pairs, mz_id_pairs + dz_id_pairs)

# Of all possible combinations, get a random sample of non-twin pairs, matching number of twin pairs
n_non_twin = len(dz_complete_pairs)
rand_inds = random.sample(range(len(non_twin_pairs)), n_non_twin)
non_twin_pairs = [non_twin_pairs[i] for i in rand_inds]

### Oscillatory Band - Spatial Overlap

Compares the spatial topography of oscillatory bands between pairs of subject. 

In [6]:
# Get database object and oscillatory band definition
db = OMDB()
osc = Osc(default=True)

In [7]:
# Compare MZ twins - Oscillatory Band Spatial Overlap

# Initialize output data variable
mz_space_dat = np.empty(shape=[0, osc.n_bands])

# Loop through MZ twin pairs, comparing them
for pair in mz_complete_pairs:
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    mz_space_dat = np.vstack([mz_space_dat, compare_spatial(dat, osc)])
    
# Get average correlation within bands across pairs
mz_avg_space = np.mean(mz_space_dat, axis=0, keepdims=True)

In [8]:
# Compare MZ twins - Oscillatory Band Spatial Overlap

# Initialize output data variable
dz_space_dat = np.empty(shape=[0, osc.n_bands])

# Loop through MZ twin pairs, comparing them
for pair in dz_complete_pairs:
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    dz_space_dat = np.vstack([dz_space_dat, compare_spatial(dat, osc)])
    
# Get average correlation within bands across pairs
dz_avg_space = np.mean(dz_space_dat, axis=0, keepdims=True)

In [9]:
# Compare non-twins - Oscillatory Band Center Frequencies

# Initialize output data variable
non_twin_space_dat = np.empty(shape=[0, osc.n_bands])

# Loop through non-twin pairs, comparing them
for pair in non_twin_pairs:
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    non_twin_space_dat = np.vstack([non_twin_space_dat, compare_spatial(dat, osc)]) 
    
# Get average correlation within bands across pairs
non_twin_avg_space = np.mean(non_twin_space_dat, axis=0, keepdims=True)

In [10]:
# Print out Results
print('MZ Twin Oscillatory Band Spatial Overlap Results: ')
print_twin_results(mz_avg_space.T, osc.bands.keys())
print('DZ Twin Oscillatory Band Spatial Overlap Results: ')
print_twin_results(dz_avg_space.T, osc.bands.keys())
print('Non-Twin Oscillatory Band Spatial Overlap Results: ')
print_twin_results(non_twin_avg_space.T, osc.bands.keys())

MZ Twin Oscillatory Band Spatial Overlap Results: 
	 Theta 	 :  0.8084
	 Alpha 	 :  0.6973
	 Beta 	 :  0.9702
	 LowGamma 	 :  0.6895
DZ Twin Oscillatory Band Spatial Overlap Results: 
	 Theta 	 :  0.7980
	 Alpha 	 :  0.7634
	 Beta 	 :  0.9082
	 LowGamma 	 :  0.6165
Non-Twin Oscillatory Band Spatial Overlap Results: 
	 Theta 	 :  0.7772
	 Alpha 	 :  0.5444
	 Beta 	 :  0.9204
	 LowGamma 	 :  0.5857


### Oscillation Parameters Comparison

The following compares oscillatory parameters, such as center frequency, power and bandwidth from within oscillatory bands. 

NOTE: As implemented, the these comparisons are not specific to how subjects are similar/different. 
Subjects can differ in either:
- The spatial topography across which the oscillation band is found and/or
- The oscillatory parameter within the vertices where the oscillatory band is found

^ This can/will be parcelled out in further analyses:
- A basic check is simply to look at the degree of overlap of oscillatory band topographies between subjects. 

In [11]:
# Initialize oscillation band object to use
osc = Osc(default=True)

# Set which oscillatory parameter to run {0: CF, 1: Power, 2: BW}
osc_param = 0

In [12]:
# Compare MZ twins - Oscillatory Band Center Frequencies

# Initialize variable to store output data
mz_corr_dat = np.zeros([len(mz_complete_pairs), osc.n_bands, 2])

# Loop through MZ twin pairs, comparing them
for ind, pair in enumerate(mz_complete_pairs):
    mz_corr_dat[ind, :, :] = compare_pair(pair, osc_param, osc)
    
# Get average correlation within bands across pairs
mz_avg_corr = np.mean(mz_corr_dat, axis=0)

In [13]:
# Compare DZ twins - Oscillatory Band Center Frequencies

# Initialize variable to store output data
dz_corr_dat = np.zeros([len(dz_complete_pairs), osc.n_bands, 2])

# Loop through DZ twin pairs, comparing them
for ind, pair in enumerate(dz_complete_pairs):
    dz_corr_dat[ind, :, :] = compare_pair(pair, osc_param, osc)
    
# Get average correlation within bands across pairs
dz_avg_corr = np.mean(dz_corr_dat, axis=0)

In [14]:
# Compare non-twins - Oscillatory Band Center Frequencies

# Initialize variable to store output data
non_twin_corr_dat = np.zeros([len(non_twin_pairs), osc.n_bands, 2])

# Loop through non-twin pairs, comparing them
for ind, pair in enumerate(non_twin_pairs):
    non_twin_corr_dat[ind, :, :] = compare_pair(pair, osc_param, osc)
    
# Get average correlation within bands across pairs
non_twin_avg_corr = np.mean(non_twin_corr_dat, axis=0)

In [15]:
# Print out results
print('MZ Twin Oscillatory Band Center Frequency Results: ')
print_twin_results(mz_avg_corr, osc.bands.keys())
print('DZ Twin Oscillatory Band Center Frequency Results: ')
print_twin_results(dz_avg_corr, osc.bands.keys())
print('Non-Twin Oscillatory Band Center Frequency Results: ')
print_twin_results(non_twin_avg_corr, osc.bands.keys())

MZ Twin Oscillatory Band Center Frequency Results: 
	 Theta 	 :  0.2032
	 Alpha 	 :  0.1547
	 Beta 	 :  0.2206
	 LowGamma 	 :  0.1826
DZ Twin Oscillatory Band Center Frequency Results: 
	 Theta 	 :  0.1717
	 Alpha 	 :  0.1142
	 Beta 	 :  0.1290
	 LowGamma 	 :  0.1047
Non-Twin Oscillatory Band Center Frequency Results: 
	 Theta 	 :  0.1114
	 Alpha 	 :  0.0955
	 Beta 	 :  0.1138
	 LowGamma 	 :  0.0449


## Slope Comparison

In [ ]:
# Check how many twin pairs are available
n_mz_pairs = len(mz_complete_pairs)
mz_corr_dat = np.zeros([n_mz_pairs, 2])

# Loop through MZ twin pairs, comparing them
for ind, pair in enumerate(mz_complete_pairs):
    mz_corr_dat[ind, :] = compare_slope(pair)

# Get average correlation within bands across pairs
mz_avg_sl_corr = np.mean(mz_corr_dat, axis=0, keepdims=True)

In [ ]:
# Check how many twin pairs are available
n_dz_pairs = len(dz_complete_pairs)
dz_corr_dat = np.zeros([n_dz_pairs, 2])

# Loop through MZ twin pairs, comparing them
for ind, pair in enumerate(dz_complete_pairs):
    dz_corr_dat[ind, :] = compare_slope(pair)

# Get average correlation within bands across pairs
dz_avg_sl_corr = np.mean(dz_corr_dat, axis=0, keepdims=True)

In [ ]:
# Compare non-twins - Oscillatory Band Center Frequencies

# Get a random sample of non-twin pairs, matching number of twin pairs
n_non_twin = n_dz_pairs
rand_inds = random.sample(range(len(non_twin_pairs)), n_non_twin)
non_twin_samp = [non_twin_pairs[i] for i in rand_inds]
non_twin_corr_dat = np.zeros([n_non_twin, 2])

# Loop through non-twin pairs, comparing them
for ind, pair in enumerate(non_twin_samp):
    non_twin_corr_dat[ind, :] = compare_slope(pair)
    
# Get average correlation within bands across pairs
non_twin_avg_sl_corr = np.mean(non_twin_corr_dat, axis=0, keepdims=True)

In [ ]:
# Print out results
print('MZ Twin Slope Results: ')
print_twin_results(mz_avg_sl_corr, ['Slope'])
print('DZ Twin Slope Results: ')
print_twin_results(dz_avg_sl_corr, ['Slope'])
print('Non-Twin Slope Results: ')
print_twin_results(non_twin_avg_sl_corr, ['Slope'])

## TEST CODE

In [ ]:
n_rand_pairs = len(rand_pairs)
rand_corr_dat = np.zeros([n_rand_pairs, 4, 2])


for ind, pair in enumerate(rand_pairs):
    rand_corr_dat[ind, :, :] = compare_twin_pair(pair)

In [ ]:
rand_pairs = [(181232, 166438), (212318, 175237), (204521, 255639),
              (191841, 293748), (214524, 352738), (223929, 111514)]

In [ ]:
pair_ind = 4

# Set subject number to load
subj_1 = mz_complete_pairs[pair_ind][0]
subj_2 = mz_complete_pairs[pair_ind][1]

In [ ]:
np.mean(mz_corr_dat, axis=0)

In [ ]:
np.mean(dz_corr_dat, axis=0)

In [ ]:
np.mean(rand_corr_dat, axis=0)

In [ ]:
np.mean(non_twin_corr_dat, axis=0)